![JohnSnowLabs](https://nlp.johnsnowlabs.com/assets/images/logo.png)

# Legal Word and Sentence Embeddings

# Legal Word and Sentence Embeddings visualization using PCA (Principal Component Analysis)

Modern NLP models work with a numerical representation of texts and their menaning. For token classification problems (inferring a class for a token, for example Name Entity Recognition) Word Embeddings are required. For sentences, paragraph, document classification - we use Sentence Embeddings.

In this notebook, we use Spark NLP Legal Word (**roberta_embeddings_legal_roberta_base**) and Sentence (**sent_bert_base_uncased_legal**) Embeddings to get those numerical representations of the semantics of the texts. The result is a 768 embeddings matrix, impossible to process by the human eye.

There are many techniques we can use to visualize those embeddings. We are using one of them - Principal Component Analysis, a dimensionality reduction process, carried out by Spark MLLib. Both embeddings have 768 dimensions, so we will reduced this dimensions from **768** to **3** (X, Y, Z) and will use a color for the word / sentence legend.

## Import Required Library

In [0]:
from johnsnowlabs import * 
import pandas as pd

# Get sample text

In [0]:
! wget -q https://raw.githubusercontent.com/JohnSnowLabs/spark-nlp-workshop/master/tutorials/Certification_Trainings/Legal/data/legal_pca_samples.csv
df = pd.read_csv('legal_pca_samples.csv')

df.head()

Out[4]:

,text,label
0,The fiscal year of the Company (herein called ...,fiscal-year
1,Each of the Borrower and each other member of ...,fiscal-year
2,Purchaser shall pay as the total Purchase Pric...,purchase-price
3,The purchase price to be paid by Purchaser to ...,purchase-price
4,The Guarantor hereby unconditionally and irrev...,guarantee


In [0]:
# Create spark dataframe
sdf = spark.createDataFrame(df)
sdf.show()

+--------------------+--------------+
 text| label|
+--------------------+--------------+
The fiscal year o...| fiscal-year|
Each of the Borro...| fiscal-year|
Purchaser shall p...|purchase-price|
The purchase pric...|purchase-price|
The Guarantor her...| guarantee|
The Holding Compa...| guarantee|
GFS will bear its...| expenses|
Each party shall ...| expenses|
Failure by either...| waiver|
Failure of any pa...| waiver|
+--------------------+--------------+

# Sentence Embeddings

In [0]:
document_assembler = nlp.DocumentAssembler()\
    .setInputCol("text")\
    .setOutputCol("document")

embeddings = nlp.BertSentenceEmbeddings.pretrained("sent_bert_base_uncased_legal", "en") \
    .setInputCols("document") \
    .setOutputCol("document_embeddings")

sent_bert_base_uncased_legal download started this may take some time.
Approximate size to download 390.8 MB
[ | ][ / ][ — ][ \ ][ | ][ / ][ — ][ \ ][ | ][ / ][ — ][OK!]

# Custom transform to retrieve the numerical embeddings from Spark NLP and pass it to Spark MLLib

In [0]:
# This class extracts the embeddings from the Spark NLP Annotation object
# from pyspark import ml as ML

class EmbeddingsUDF(
    Transformer, ML.param.shared.HasInputCol,  ML.param.shared.HasOutputCol,
    ML.util.DefaultParamsReadable, ML.util.DefaultParamsWritable
):
    @keyword_only
    def __init__(self):
        super(EmbeddingsUDF, self).__init__()

        def _sum(r):
            result = 0.0
            for e in r:
                result += e
            return result

        self.udfs = {
            'convertToVectorUDF': F.udf(lambda vs: ML.linalg.Vectors.dense(vs), ML.linalg.VectorUDT()),
            'sumUDF': F.udf(lambda r: _sum(r), T.FloatType())
        }

    def _transform(self, dataset):

        results = dataset.select(
            "*", F.explode("document_embeddings.embeddings").alias("embeddings")
        )
        results = results.withColumn(
            "features",
            self.udfs['convertToVectorUDF'](F.col("embeddings"))
        )
        results = results.withColumn(
            "emb_sum",
            self.udfs['sumUDF'](F.col("embeddings"))
        )
        # Remove those with embeddings all zeroes (so we can calculate cosine distance)
        results = results.where(F.col("emb_sum")!=0.0)

        return results

In [0]:
embeddings_for_pca = EmbeddingsUDF()

In [0]:
DIMENSIONS  = 3

In [0]:
# import pyspark
pca =ML.feature.PCA(k=DIMENSIONS, inputCol="features", outputCol="pca_features")

### Full Spark NLP + Spark MLLib pipeline

In [0]:
# We did all process in one pipeline

pipeline = Pipeline().setStages([document_assembler, embeddings, embeddings_for_pca, pca])

In [0]:
pipeline.getStages()

Out[12]: [DocumentAssembler_ebcd11e58f87,
 BERT_SENTENCE_EMBEDDINGS_dae49a767331,
 EmbeddingsUDF_b0b89b246cb5,
 PCA_591fe878e9d2]

In [0]:
model = pipeline.fit(sdf)

In [0]:
result = model.transform(sdf)

In [0]:
result.select('pca_features', 'label').show(truncate=False)

+------------------------------------------------------------+--------------+
pca_features |label |
+------------------------------------------------------------+--------------+
[-11.77247850164304,-3.1899495027460802,4.4911347376512625] |fiscal-year |
[-11.401136121266068,-3.7697374004104334,3.2405296650257367]|fiscal-year |
[-4.783311752658829,-0.4942463142576333,2.8697524728283947] |purchase-price|
[-5.45598430275419,-1.3412475150984924,3.3317012364357033] |purchase-price|
[-8.841678740319807,-1.8203554687294219,0.13389973609380276]|guarantee |
[-11.532879448636747,-2.49956446707214,0.8141248995964393] |guarantee |
[-5.731717033815797,-3.8158663928138394,3.5135639094225213] |expenses |
[-3.801054731057207,-4.345368492542858,1.6774270432318936] |expenses |
[-6.7835365507704735,-5.815450877483512,3.236137726083089] |waiver |
[-7.139197100846756,-6.340437220099111,1.2347006100719193] |waiver |
+------------------------------------------------------------+--------------+

In [0]:
df = result.select('pca_features', 'label').toPandas()

df
# As you see, dimension values are inside a list

Out[16]:

,pca_features,label
0,"[-11.77247850164304, -3.1899495027460802, 4.49...",fiscal-year
1,"[-11.401136121266068, -3.7697374004104334, 3.2...",fiscal-year
2,"[-4.783311752658829, -0.4942463142576333, 2.86...",purchase-price
3,"[-5.45598430275419, -1.3412475150984924, 3.331...",purchase-price
4,"[-8.841678740319807, -1.8203554687294219, 0.13...",guarantee
5,"[-11.532879448636747, -2.49956446707214, 0.814...",guarantee
6,"[-5.731717033815797, -3.8158663928138394, 3.51...",expenses
7,"[-3.801054731057207, -4.345368492542858, 1.677...",expenses
8,"[-6.7835365507704735, -5.815450877483512, 3.23...",waiver
9,"[-7.139197100846756, -6.340437220099111, 1.234...",waiver


In [0]:
# We extract the dimension values out off the list

df["x"] = df["pca_features"].apply(lambda x: x[0])

df["y"] = df["pca_features"].apply(lambda x: x[1])

df["z"] = df["pca_features"].apply(lambda x: x[2])

df = df[["x", "y", "z", "label"]]

df

Out[17]:

,x,y,z,label
0,-11.772479,-3.189950,4.491135,fiscal-year
1,-11.401136,-3.769737,3.240530,fiscal-year
2,-4.783312,-0.494246,2.869752,purchase-price
3,-5.455984,-1.341248,3.331701,purchase-price
4,-8.841679,-1.820355,0.133900,guarantee
5,-11.532879,-2.499564,0.814125,guarantee
6,-5.731717,-3.815866,3.513564,expenses
7,-3.801055,-4.345368,1.677427,expenses
8,-6.783537,-5.815451,3.236138,waiver
9,-7.139197,-6.340437,1.234701,waiver


In [0]:
import plotly.express as px

fig = px.scatter_3d(df, x='x', y='y', z='z', color='label', width=800, height=600)

fig.show()

# Word Embeddings

We can also visualize the semantics of words, instead of full texts, by using Word Embeddings. We will add a Tokenizer and a WordEmbeddings model to get those embeddings, and them apply PCA as before.

In [0]:
document_assembler = nlp.DocumentAssembler()\
    .setInputCol("text")\
    .setOutputCol("document")

tokenizer = nlp.Tokenizer() \
    .setInputCols("document")\
    .setOutputCol("token")

embeddings = nlp.RoBertaEmbeddings.pretrained("roberta_embeddings_legal_roberta_base","en") \
    .setInputCols(["document", "token"])\
    .setOutputCol("document_embeddings")

roberta_embeddings_legal_roberta_base download started this may take some time.
Approximate size to download 447.2 MB
[ | ][ / ][ — ][ \ ][ | ][ / ][ — ][ \ ][ | ][ / ][ — ][OK!]

In [0]:
# Firstly we splitted the pipeline in two to get all token embeddings

pipeline = Pipeline().setStages([document_assembler, tokenizer, embeddings])

In [0]:
model = pipeline.fit(sdf)

In [0]:
result = model.transform(sdf)

In [0]:
result_df = result.select("label", F.explode(F.arrays_zip(result.token.result, result.document_embeddings.embeddings)).alias("cols"))\
                   .select(F.expr("cols['0']").alias("token"),
                           F.expr("cols['1']").alias("embeddings"),
                           "label")

result_df.show(truncate = 80)


+--------+--------------------------------------------------------------------------------+-----------+
 token| embeddings| label|
+--------+--------------------------------------------------------------------------------+-----------+
 The|[-0.19058336, 0.029071666, 0.13235594, 0.19562267, 0.77783304, 0.28990862, -0...|fiscal-year|
 fiscal|[-0.19621758, 0.1450966, 0.23111373, -0.5060171, -0.38397145, -0.16950944, 0....|fiscal-year|
 year|[0.080062926, 0.22008324, 0.23202527, -0.4419723, 0.5893601, -0.2369214, 0.14...|fiscal-year|
 of|[-0.14090309, 0.15613593, 0.240002, -0.24493237, 0.8977565, 0.4878101, 0.0917...|fiscal-year|
 the|[-0.060954966, -0.082321286, 0.31499293, 0.12840006, -0.014587007, 0.9788801,...|fiscal-year|
 Company|[-0.060746796, 0.27488375, 0.07146084, -0.39569968, 0.7331524, 0.805157, 0.05...|fiscal-year|
 (|[0.1109849, -0.23188409, 0.112354815, 0.07458246, 0.86609805, -0.11237953, -0...|fiscal-year|
 herein|[-0.080648616, -0.058289625, 0.0014471561, -0.014948197, 0.2739821, -0.203156...|fiscal-year|
 called|[0.05355736, 0.15397248, 0.060474757, 0.43707842, 1.2082461, 0.19268245, 0.22...|fiscal-year|
 the|[-0.08710325, 0.07147762, 0.15197307, 0.064896114, -0.081882775, 0.29937804, ...|fiscal-year|
 "|[0.055207547, 0.19191705, 0.009991348, -0.0025674403, 1.0418487, 0.2136438, -...|fiscal-year|
 fiscal|[-0.14709541, 0.017821632, 0.27213323, -0.4875228, -0.48709244, -0.22528088, ...|fiscal-year|
 year|[0.09589225, -0.01859519, 0.2457679, -0.52658224, 0.23036297, -0.16599059, 0....|fiscal-year|
 ")|[0.07879877, -0.059296403, -0.09431644, 0.22301325, 0.49477533, -0.19168587, ...|fiscal-year|
 shall|[-0.18689293, -0.030555628, 0.24993141, -0.08885709, -0.15760188, -0.33531097...|fiscal-year|
 end|[0.15657806, -0.16092363, 0.25242162, 0.05500044, -2.0781853, -0.21233709, 0....|fiscal-year|
 on|[0.3785015, 0.45011446, 0.36284545, 0.48866254, 0.71298033, -0.018029578, 0.2...|fiscal-year|
December|[0.46872914, 0.040406942, -0.02183514, 0.15274873, 1.0853438, 0.26045802, 0.1...|fiscal-year|
 31|[0.20654199, 0.15854527, 0.18410307, 0.081837796, 0.739096, -0.01880616, -0.0...|fiscal-year|
 of|[0.043495774, -0.1159763, 0.09074756, -0.10574566, 1.0775033, 0.43243805, 0.0...|fiscal-year|
+--------+--------------------------------------------------------------------------------+-----------+
only showing top 20 rows

In [0]:
# Here we defined inheritance class from that defined previously EmbeddingsUDF class
class WordEmbeddingsUDF(EmbeddingsUDF):    
    def _transform(self, dataset):
        
        results = dataset.select('token', 'label', 'embeddings') # We changed this line because our embedding cloumn is already exploded

        results = results.withColumn(
            "features",
            self.udfs['convertToVectorUDF'](F.col("embeddings"))
        )
        results = results.withColumn(
            "emb_sum",
            self.udfs['sumUDF'](F.col("embeddings"))
        )
        # Remove those with embeddings all zeroes (so we can calculate cosine distance)
        results = results.where(F.col("emb_sum")!=0.0)

        return results

In [0]:
embeddings_for_pca = WordEmbeddingsUDF()

In [0]:
DIMENSIONS  = 3

In [0]:
# import pyspark
pca = ML.feature.PCA(k=DIMENSIONS, inputCol="features", outputCol="pca_features")

## Full Spark NLP + Spark MLLib pipeline

In [0]:
# We run the second part of the pipeline

pipeline = Pipeline().setStages([embeddings_for_pca, pca])


In [0]:
model = pipeline.fit(result_df)

In [0]:
result = model.transform(result_df)

In [0]:
result.select("token", "embeddings", "pca_features", "label").show(truncate = 60)

+--------+------------------------------------------------------------+------------------------------------------------------------+-----------+
 token| embeddings| pca_features| label|
+--------+------------------------------------------------------------+------------------------------------------------------------+-----------+
 The|[-0.19058336, 0.029071666, 0.13235594, 0.19562267, 0.7778...| [6.57477032855015,7.652863050357908,-0.9220377602685892]|fiscal-year|
 fiscal|[-0.19621758, 0.1450966, 0.23111373, -0.5060171, -0.38397...| [5.227399434004428,-0.06940175895114242,-1.397651866076717]|fiscal-year|
 year|[0.080062926, 0.22008324, 0.23202527, -0.4419723, 0.58936...| [9.185601550283767,1.0014787375644385,1.0998377664580117]|fiscal-year|
 of|[-0.14090309, 0.15613593, 0.240002, -0.24493237, 0.897756...| [6.397932240279675,3.1917345438580775,-1.0405448987305215]|fiscal-year|
 the|[-0.060954966, -0.082321286, 0.31499293, 0.12840006, -0.0...| [7.772305994459045,8.005478320815016,-2.2717379067281924]|fiscal-year|
 Company|[-0.060746796, 0.27488375, 0.07146084, -0.39569968, 0.733...| [3.20136574431197,3.9378578166983607,-2.4062436304322175]|fiscal-year|
 (|[0.1109849, -0.23188409, 0.112354815, 0.07458246, 0.86609...| [3.2695697877419923,1.5800763788269503,2.1269484316219844]|fiscal-year|
 herein|[-0.080648616, -0.058289625, 0.0014471561, -0.014948197, ...| [5.247827031337183,0.5408448172923814,-0.4720354131608218]|fiscal-year|
 called|[0.05355736, 0.15397248, 0.060474757, 0.43707842, 1.20824...| [4.8478980999428725,1.4702124121479105,0.5524321998206301]|fiscal-year|
 the|[-0.08710325, 0.07147762, 0.15197307, 0.064896114, -0.081...| [6.271759722645766,5.41234138442504,-0.6796094771921956]|fiscal-year|
 "|[0.055207547, 0.19191705, 0.009991348, -0.0025674403, 1.0...| [2.4479710107088857,1.531854886543407,-0.10306184912496506]|fiscal-year|
 fiscal|[-0.14709541, 0.017821632, 0.27213323, -0.4875228, -0.487...| [5.302239166045237,-0.1566938257568297,-1.4091704578070436]|fiscal-year|
 year|[0.09589225, -0.01859519, 0.2457679, -0.52658224, 0.23036...| [9.220590934785625,1.3564938894764567,1.429932310792168]|fiscal-year|
 ")|[0.07879877, -0.059296403, -0.09431644, 0.22301325, 0.494...| [1.9428333289018993,1.5016632043673583,2.006954414833621]|fiscal-year|
 shall|[-0.18689293, -0.030555628, 0.24993141, -0.08885709, -0.1...| [5.095635492349415,6.438950836712824,0.7663067830509055]|fiscal-year|
 end|[0.15657806, -0.16092363, 0.25242162, 0.05500044, -2.0781...| [6.9425999339924855,1.3926470503804107,-0.3101335040098114]|fiscal-year|
 on|[0.3785015, 0.45011446, 0.36284545, 0.48866254, 0.7129803...| [7.604271142867041,3.464276899108001,1.311439926314339]|fiscal-year|
December|[0.46872914, 0.040406942, -0.02183514, 0.15274873, 1.0853...| [6.591853469712691,2.656919429265407,-3.1115202407232023]|fiscal-year|
 31|[0.20654199, 0.15854527, 0.18410307, 0.081837796, 0.73909...| [4.474304955232831,1.8455628872285075,0.5983483498703467]|fiscal-year|
 of|[0.043495774, -0.1159763, 0.09074756, -0.10574566, 1.0775...|[6.018187928089208,3.1989201426425917,-0.014781006381167364]|fiscal-year|
+--------+------------------------------------------------------------+------------------------------------------------------------+-----------+
only showing top 20 rows

In [0]:
df = result.select('token', 'pca_features',  'label').toPandas()

df

Out[33]:

,token,pca_features,label
0,The,"[6.57477032855015, 7.652863050357908, -0.92203...",fiscal-year
1,fiscal,"[5.227399434004428, -0.06940175895114242, -1.3...",fiscal-year
2,year,"[9.185601550283767, 1.0014787375644385, 1.0998...",fiscal-year
3,of,"[6.397932240279675, 3.1917345438580775, -1.040...",fiscal-year
4,the,"[7.772305994459045, 8.005478320815016, -2.2717...",fiscal-year
...,...,...,...
672,constitute,"[6.129942369461009, -1.5722390954279102, -0.58...",waiver
673,a,"[8.76467750087386, 1.0030860430885131, 0.45322...",waiver
674,continuing,"[4.150359211031833, -0.772288174067816, -1.152...",waiver
675,waiver,"[7.824426810612914, -1.474974474970483, -3.411...",waiver


In [0]:
df["x"] = df["pca_features"].apply(lambda x: x[0])

df["y"] = df["pca_features"].apply(lambda x: x[1])

df["z"] = df["pca_features"].apply(lambda x: x[2])

df = df[["token", "x", "y", "z", "label"]]

df

Out[34]:

,token,x,y,z,label
0,The,6.574770,7.652863,-0.922038,fiscal-year
1,fiscal,5.227399,-0.069402,-1.397652,fiscal-year
2,year,9.185602,1.001479,1.099838,fiscal-year
3,of,6.397932,3.191735,-1.040545,fiscal-year
4,the,7.772306,8.005478,-2.271738,fiscal-year
...,...,...,...,...,...
672,constitute,6.129942,-1.572239,-0.582969,waiver
673,a,8.764678,1.003086,0.453227,waiver
674,continuing,4.150359,-0.772288,-1.152193,waiver
675,waiver,7.824427,-1.474974,-3.411840,waiver


In [0]:
import plotly.express as px

fig = px.scatter_3d(df, x = 'x', y = 'y', z = 'z', color = "label", width=1000, height = 800, hover_data = ["token", "label"])

fig.show()

That chart is super cool because you can see how the same token gets different embeddings depending on the context.